In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, END, START
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from dotenv import load_dotenv
import os
import json

load_dotenv()


class State(TypedDict):
    has_pet: bool | None
    animal: str | None
    pet_name: str | None
    messages: Annotated[list[BaseMessage], add_messages]
    

def ask_about_pet(state: State):
    if not state.get("messages", []):
        return {"has_pet": None, "messages": [AIMessage(content="Hi there! Do you have any pets?")]}
        
    system_prompt = {
        "role": "system",
        "content": """You are a friendly assistant trying to determine if the user has a pet,
        and if so what kind of animal it is and the name of the pet. 
        You must always respond in **valid JSON format** with 3 keys: 
        - 'has_pet': true if they have a pet, false if they don't, null if undecided
        - 'animal': The type of animal the pet is, eg dog, cat, hamster, null if unknown
        - 'pet_name': The pet's name or null if unknown
        - 'message': Your conversational reply to keep the discussion going
        
        IMPORTANT:
        - Do NOT return anything except JSON. Do NOT include extra text, explanations, or formatting.
        - Once you have determined they don't have a pet, give a final friendly closing message and say goodbye. 
          Do not ask further questions. 
        - Once you have determined they don't have a pet, what kind of animal the pet is, and the pet's name, 
          give a final friendly closing message and say goodbye. Do not ask further questions.
        - If you cannot determine if they have a pet, ask another natural question to clarify.
        
        Example responses:
        {"has_pet": true, "animal": "hamster", "pet_name": "Spot", "message": "Spot sounds wonderful! Thanks for sharing!"}
        {"has_pet": true, "animal": "dog", "pet_name": null, "message": "Great what's your dog's name?"}
        {"has_pet": false, "animal": null, "pet_name": null, "message": "Alright, no worries! Thanks for letting me know. Goodbye!"}
        {"has_pet": null, "animal": null, "pet_name": null, "message": "Interesting! Do have any animals in your home?"}
        """
    }
    
    messages = [system_prompt] + state.get("messages", [])
    response = llm.invoke(messages)
    try:
        response_dict = json.loads(response.content)
        return {
            "has_pet": response_dict.get("has_pet"),
            "animal": response_dict.get("animal"),
            "pet_name": response_dict.get("pet_name"),
            "messages": [AIMessage(content=response_dict.get("message"))]
        }
    except Exception as e:
        print(response)
        return {"messages": [AIMessage(content="Oops, something went wrong. Can you try rephrasing that?")]}


def is_finished(state: State) -> bool:
    match state:
        case {"has_pet": False}:
            return True
        case {"has_pet": True, "animal": str(), "pet_name": str()}:
            return True
        case _:
            return False


def run_chatbot():
    openai_api_key = os.getenv("OPENAI_API_KEY")
    llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=openai_api_key)
    graph_builder = StateGraph(State)
    graph_builder.add_node("ask_about_pet", ask_about_pet)
    graph_builder.add_edge(START, "ask_about_pet")
    graph_builder.add_edge("ask_about_pet", END)
    memory = MemorySaver()
    graph = graph_builder.compile(checkpointer=memory)

    config = {"configurable": {"thread_id": "1"}}
    state = {"has_pet": None, "animal": None, "pet_name": None, "messages": []}
            
    # The bot starts the conversation
    for event in graph.stream(state, config):
        for value in event.values():
            print("Bot:", value["messages"][-1].content)
            state = value
            break

    while not is_finished(state):
        try:
            user_input = input("User: ")
            if user_input.lower() in ["quit", "exit", "q", "bye", ""]:
                print("Goodbye!")
                break
            state["messages"] = add_messages(state.get("messages", []), [HumanMessage(content=user_input)])
            for event in graph.stream(state, config):
                for value in event.values():
                    print("Bot:", value["messages"][-1].content)
                    state = value
                    break
        except Exception as e:
            print(" Something went wrong. Goodbye!")
            print(e)
            break
    

if __name__ == "__main__":
    run_chatbot()